# Step 5: Add a deployment pipeline
In previous four steps you implemented an automated data processing and model building pipeline. Each run of the pipeline produces a new version of the model. This notebook implements the automated model deployment step in our ML workflow.

You can use a [SageMaker MLOps project template](https://docs.aws.amazon.com/sagemaker/latest/dg/sagemaker-projects-templates.html) to provision a ready-to use model deployment CI/CD pipeline.

This template automates the deployment of models in the SageMaker model registry to SageMaker endpoints for real-time inference. This template recognizes changes in the model registry. When a new model version is registered and approved, it automatically initiates a deployment.

![](img/six-steps-5.png)

In [ ]:
import boto3
import sagemaker 
from time import gmtime, strftime, sleep

sagemaker.__version__

In [ ]:
%store -r 

%store

try:
    initialized
except NameError:
    print("+++++++++++++++++++++++++++++++++++++++++++++++++")
    print("[ERROR] YOU HAVE TO RUN 00-start-here notebook   ")
    print("+++++++++++++++++++++++++++++++++++++++++++++++++")

In [ ]:
sm = boto3.client("sagemaker")

## Create an MLOps project
Follow the same procedure as in the step 4 notebook to create a model deployment MLOps project. 

### Option 1: Create project programmatically
Use `boto3` to create an MLOps project via a SageMaker API.

In [ ]:
sc = boto3.client("servicecatalog")

sc_provider_name = "Amazon SageMaker"
sc_product_name = "MLOps template for model deployment"

In [ ]:
p_ids = [p['ProductId'] for p in sc.search_products(
    Filters={
        'FullTextSearch': [sc_product_name]
    },
)['ProductViewSummaries'] if p["Name"]==sc_product_name]

In [ ]:
p_ids

In [ ]:
# If you get any exception from this code, go to the Option 2 and create a project in Studio UX
if not len(p_ids):
    raise Exception("No Amazon SageMaker ML Ops products found!")
elif len(p_ids) > 1:
    raise Exception("Too many matching Amazon SageMaker ML Ops products found!")
else:
    product_id = p_ids[0]
    print(f"ML Ops product id: {product_id}")

In [ ]:
provisioning_artifact_id = sorted(
    [i for i in sc.list_provisioning_artifacts(
        ProductId=product_id
    )['ProvisioningArtifactDetails'] if i['Guidance']=='DEFAULT'],
    key=lambda d: d['Name'], reverse=True)[0]['Id']

In [ ]:
provisioning_artifact_id

In [ ]:
project_name = f"model-deploy-{strftime('%-m-%d-%H-%M-%S', gmtime())}"

In [ ]:
project_parameters = [
    {
        'Key': 'SourceModelPackageGroupName',
        'Value': model_package_group_name
    },
]

Finally, create a SageMaker project from the service catalog product template:

In [ ]:
# create SageMaker project
r = sm.create_project(
    ProjectName=project_name,
    ProjectDescription="Model build project",
    ServiceCatalogProvisioningDetails={
        'ProductId': product_id,
        'ProvisioningArtifactId': provisioning_artifact_id,
        'ProvisioningParameters': project_parameters
    },
)

print(r)
project_id = r["ProjectId"]

<div class="alert alert-info"> 💡 <strong> Wait until project creation is completed </strong>
</div>

In [ ]:
while sm.describe_project(ProjectName=project_name)['ProjectStatus'] != 'CreateCompleted':
    sleep(10)
    print("Waiting for project creation completion")

print(f"MLOps project {project_name} creation completed")

### Option 2: Create a project in Studio
<div class="alert alert-info"> 💡 <strong> Skip this section if you created a project programmatically </strong>

To create a project in Studio:

1. In the Studio sidebar, choose the **Home** icon.
2. Select **Deployments** from the menu, and then select **Projects**.
3. Choose **Create project**.
    - The **Create project** tab opens displaying a list of available project templates.
4. For **SageMaker project templates**, choose **SageMaker templates**. 
5. Choose **MLOps template for model deployment**
6. Choose **Select project template**.

![](img/create-mlops-project.png)

![](img/create-mlops-project-deploy.png)

The **Create project** tab changes to display **Project details**.
Enter the following information:
1. Project name. Note the name requirements
2. Project description, optional
3. Model package group name - enter your model package group name you used in the step 3 and 4 notebooks. It is `from-idea-to-prod-model-group` if you used the default one

Optionally, add tags, which are key-value pairs that you can use to track your projects.

![](img/project-details-deploy.png)

Choose **Create project** and wait for the project to appear in the Projects list.

## Working with MLOps project for model deployment
The template provisions a CodeCommit repository with configuration files to specify the model deployment steps, AWS CloudFormation templates to define endpoints as infrastructure, and seed code for testing the endpoint.

This template provides the following resources:

1. An AWS CodeCommit repository that contains sample code that deploys models to endpoints in staging and production environments
2. An AWS CodePipeline pipeline that has `source`, `build`, `deploy-to-staging`, and `deploy-to-production` steps. The `source` step points to the CodeCommit repository, and the `build` step gets the code from that repository and generates CloudFormation stacks to deploy. The `deploy-to-staging` and `deploy-to-production` steps deploy the CloudFormation stacks to their respective environments. There is a manual approval step between the staging and production build steps, so that a MLOps engineer must approve the model before it is deployed to production.
3. An Amazon EventBridge rule to launch a CodePipeline pipeline execution when a model package version is approved or rejected.
4. There is also a manual approval step after the placeholder unit tests. You can implement your own tests to replace the placeholders tests.

The template also deploys an Amazon S3 bucket to store artifacts, including CodePipeline and CodeBuild artifacts, and any artifacts generated from the SageMaker pipeline runs.

The following diagram shows the architecture.

<img src="img/mlops-model-deploy.png" width="600"/>

You don't need to implement any configuration changes for the project. The model deployment pipeline works out of the box.
To start the model deployment pipeline, you must approve the model version in the model registry.

### Approve the model version
Approving the model version causes the MLOps system to launch the model deployment process. First, the model deployment pipeline deploys the model version to staging.

You can approve the model version either in Studio in the Model registry or do it programmatically in the notebook. Let's do it programatically.

In [ ]:
model_package_group_name

In [ ]:
# list all model packages and select the latest one
model_packages = []

for p in sm.get_paginator('list_model_packages').paginate(
        ModelPackageGroupName=model_package_group_name,
        SortBy="CreationTime",
        SortOrder="Descending",
    ):
    model_packages.extend(p["ModelPackageSummaryList"])

if len(model_packages) == 0:
    raise Exception(f"No model package is found for {model_package_group_name} model package group")
    
latest_model_package_arn = model_packages[0]["ModelPackageArn"]
print(latest_model_package_arn)

The following statement sets the `ModelApprovalStatus` for the most recent model package in the model registry to `Approved`. The model package state change launches the EventBridge rule and the rule launches the CodePipeline CI/CD pipeline with model deployment.

In [ ]:
model_package_update_response = sm.update_model_package(
    ModelPackageArn=latest_model_package_arn,
    ModelApprovalStatus="Approved",
)

### Deployment pipeline execution
The model deployment CI/CD pipeline performs the following actions:
1. Create a SageMaker endpoint with the name `<PROJECT-NAME>-staging` in the current account
2. Run the test script on the staging endpoint
3. Wait until the test result is manually approved in [AWS CodePipeline console](https://console.aws.amazon.com/codesuite/codepipeline)
4. Create a SageMaker endpoint with the name `<PROJECT-NAME>-prod` in the current account

Wait about 10-15 minutes until the pipeline deploys the staging endpoint. You can see the status of the endpoint in **Home** > **Deployments** > **Endpoints**:

![](img/endpoint-staging.png)

After the endpoint status changed from `Creating` to `InService`, you can launch the model deployment process to the production stage by manually approving the **DeployStaging** stage of the CodePipeline pipeline.

### Deploy the model version to production
Let's construct the approval link. 
If you used the option 1 `boto3` to create an MLOps project, the `project_name` and `project_id` are set automatically. You can run the following code cell to print the values. If you followed the UX instructions to create a project, you must set the `project_name` manually.

In [ ]:
try:
    print(project_name)
    print(project_id)
except NameError:
    print("++++++++++++++++++++++++++++++++++++++")
    print("You must set the project_name manually")
    print("++++++++++++++++++++++++++++++++++++++")

In [ ]:
# Set to the model deployment project name if you didn't use boto3-based deployment
# project_name = "<USE YOUR PROJECT NAME>"

# Get project id
project_id = sm.describe_project(ProjectName=project_name)['ProjectId']

# Construct the CodePipline pipeline name
code_pipeline_name = f"sagemaker-{project_name}-{project_id}-modeldeploy"

In [ ]:
from IPython.core.display import display, HTML

# Show the approval link
display(
    HTML(
        '<b>Please approve the manual step in <a target="top" href="https://console.aws.amazon.com/codesuite/codepipeline/pipelines/{}/view?region={}">AWS CodePipeline</a></b>'.format(
            code_pipeline_name, region)
    )
)

Click on the link ^^^ above ^^^ to open the CodePipeline console with the pipeline execution workflow.

In the **DeployStaging stage**, choose **Review** on the **ApproveDeployment** step. In the **Review** dialog box, choose **Approve**.

Approving the **DeployStaging** stage causes the deployment pipeline to deploy the model to production. To view the endpoint, choose the **Deployments** > **Endpoints** in **Home** in Studio.

After a successful completion of the CI/CD pipeline, you see two endpoints in status `InService` in **SageMaker resources** > **Endpoints**:

![](img/endpoint-prod.png)

You can also see that both endpoints, `staging` and `prod`, are visible in the deployment project:

![](img/project-endpoints.png)


## Summary
In this notebook you implemented an automated CI/CD deployment pipeline with the following features:
- use CloudFormation IaC templates for SageMaker real-time inference endpoint deployment
- model approval in the model registry launches the model deployment pipeline
- model deployment pipeline contains two stages, staging and production with automated tests for the staging endpoint and manual approval for the production deployment

## Continue with the step 6
open the step 6 [notebook](06-monitoring.ipynb).

## Further development ideas for your real-world projects
- Add end-to-end data encryption using AWS KMS keys
- Create a [custom SageMaker project template](https://docs.aws.amazon.com/sagemaker/latest/dg/sagemaker-projects-templates-custom.html) for model deployment to cover your specific project requirements
- Add [multi-account model deployment](https://aws.amazon.com/blogs/machine-learning/multi-account-model-deployment-with-amazon-sagemaker-pipelines/) to your ML workflow
- Add automated model tests to the placeholder in the CodePipeline pipeline
- Use [Amazon SageMaker Inference Recommender](https://docs.aws.amazon.com/sagemaker/latest/dg/inference-recommender.html) to run automated load tests for your inference endpoints and to select the best instance type and configuration

## Additional resources
- [Deploy a Machine Learning Model to a Real-Time Inference Endpoint](https://aws.amazon.com/getting-started/hands-on/machine-learning-tutorial-deploy-model-to-real-time-inference-endpoint/)
- [SageMaker MLOps Project Walkthrough](https://docs.aws.amazon.com/sagemaker/latest/dg/sagemaker-projects-walkthrough.html)
- [Amazon SageMaker Pipelines lab in SageMaker Immersion Day](https://catalog.us-east-1.prod.workshops.aws/workshops/63069e26-921c-4ce1-9cc7-dd882ff62575/en-US/lab6)
- [Amazon SageMaker secure MLOps](https://github.com/aws-samples/amazon-sagemaker-secure-mlops)
- [Testing approaches for Amazon SageMaker ML models](https://aws.amazon.com/blogs/machine-learning/testing-approaches-for-amazon-sagemaker-ml-models/)
- [Model hosting patterns in Amazon SageMaker blog series](https://aws.amazon.com/blogs/machine-learning/model-hosting-patterns-in-amazon-sagemaker-part-1-common-design-patterns-for-building-ml-applications-on-amazon-sagemaker/)
- [Take advantage of advanced deployment strategies using Amazon SageMaker deployment guardrails](https://aws.amazon.com/blogs/machine-learning/take-advantage-of-advanced-deployment-strategies-using-amazon-sagemaker-deployment-guardrails/)
- [MLOps deployment best practices for real-time inference model serving endpoints with Amazon SageMaker](https://aws.amazon.com/blogs/machine-learning/mlops-deployment-best-practices-for-real-time-inference-model-serving-endpoints-with-amazon-sagemaker/)

# Shutdown kernel

In [ ]:
%%html

<p><b>Shutting down your kernel for this notebook to release resources.</b></p>
<button class="sm-command-button" data-commandlinker-command="kernelmenu:shutdown" style="display:none;">Shutdown Kernel</button>
        
<script>
try {
    els = document.getElementsByClassName("sm-command-button");
    els[0].click();
}
catch(err) {
    // NoOp
}    
</script>